In [1]:
import tensorflow as tf
import tensorflow.keras.layers as tfl
from tensorflow.keras import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.vgg16 import VGG16

from IPython.display import HTML, display
import tabulate

import numpy as np
import pandas as pd

from PIL import Image
import matplotlib.pyplot as plt

# Incremental learning
## Move-to-data

In [3]:
test_datagen = ImageDataGenerator(rescale = 1.0/255.)
test_generator = test_datagen.flow_from_directory('../DB/test', batch_size = 3568, class_mode = 'categorical', target_size = (im.width, im.height))

# On récupère les images d'un batch de la taille du dataset
i = 0
for image, y in test_generator:
    if i == 0: break # sinon le generator continue sa boucle (??)

NameError: name 'im' is not defined

In [ ]:
model = tf.keras.models.load_model('../models/best_models', custom_objects=None, compile=True, options=None)

In [ ]:
model.summary()

In [ ]:
model.evaluate(image, y)

In [ ]:
output = [layer.output for layer in model.layers]

In [ ]:
move_model = Model(output[0], [output[-2], output[-1]])

$$w_j′=w_j+(||w_j||∗\frac{v_i}{||v_i||} − w_j)*\epsilon$$

In [ ]:
def move_to_data(v, w):
    epsilon = 0.0001
    new_w = w + (tf.norm(w) * (v / tf.norm(v)) - w) * epsilon
    return new_w

In [ ]:
im = Image.open('../DB/test/Bowl/BowlPlace6Subject4_2_bboxes_157_0.png')
plt.imshow(im)

In [ ]:
for i in range(len(image)):
    a, _ = move_model(tf.expand_dims(np.array(im), axis=0))
    v = a
    w, b = model.get_layer('dense_3').get_weights()
    
    new_weights = np.zeros(w.shape)
    for j in range(5):
        el_w = w[:, j]
        el_new_w = move_to_data(v, el_w)
        new_weights[:, j] = el_new_w

    model.get_layer('dense_3').set_weights([new_weights, b])

In [ ]:
model.evaluate(image, y)

## Fine tuning

In [ ]:
for layer in model.layers[:-1]:
    layer.trainable = False
    
for layer in model.layers:
    print(layer, layer.trainable)

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=[
    'accuracy', 
    'categorical_accuracy',
    tf.keras.metrics.Recall(),
    tf.keras.metrics.Precision()])

In [ ]:
model.fit(image, y, epochs=5)

In [ ]:
model.evaluate(image, y)